# Практика для метрик классификации

## Практика 1 (бинарная классификация):

Реализуем бинарную классификацию записей голосов (мужских и женских) на основе характеристик записей их разговоров.

В предложенном датасете есть ряд свойств, определенных по аудиозаписям:

- meanfreq: средняя частота голоса (в кГц)
- sd: стандартное отклонение частоты голоса
- median: медианная частота (в кГц)
- Q25: значение в первом квартиле (в кГц)
- Q75: значение в третьем квартиле (в кГц)
- IQR: интерквартильный размах (в кГц)
- skew: ассиметрия
- kurt: эксцесс
- sp.ent: спектральная энтропия
- sfm: энтропия Винера
- mode: мода частоты
- centroid: частотный центроид
- meanfun: средняя основная частота, измеренная по акустическому сигналу
- minfun:  минимальная основная частота, измеренная по акустическому сигналу
- maxfun: максимальная основная частота, измеренная в акустическом сигнале
- meandom: среднее значение доминирующей частоты, измеренной по акустическому сигналу
- mindom: минимум доминирующей частоты, измеренной в акустическом сигнале
- maxdom: максимум доминирующей частоты, измеренной в акустическом сигнале
- dfrange: диапазон доминантных частот, измеренное на звуковой сигнал
- modindx: индекс модуляции голоса

Прежде, чем проводить анализ, можно с помощью визуализации оценить, есть ли различия в частотах голосов мужчин и женщин. Можем построить распределения, например, средних частот:

![voices](./img/classification_practice.png)

Красному цвету соответствуют данные для женских голосов, голубому — для мужских. В целом можно отметить, что распределение частот женских голосов сдвинуто вправо относительно распределения мужских.

Итак, характеристики для голосов различаются, попробуем теперь реализовать классификацию на их основе.


## Задание

Попробуйте построить модель, предсказывающую пол обладателя записи голоса.

Для этого:

Разделите выборку на обучающую и тренировочную с параметрами `test_size=0.3, random_state=42`.

Нормализуйте признаки с помощью функции `StandardScaler()`. Учитывайте, что нормализация тестовой выборки производится по среднему и отклонению тренировочной, которую мы считаем репрезентативной относительно генеральной совокупности.

Обучите модель логистической регрессии на подготовленных данных.

### Пояснение
Тестовые данные не должны влиять на параметры нормализации. Нужно использовать `SCALER.TRANSFORM` вместо `SCALER.FIT_TRANSFORM`, чтобы применять параметры нормализации, рассчитанные для тренировочных данных. Иначе данные в трейне и в тесте будут нормализованы по - разному.

In [1]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score 
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score 
from sklearn.metrics import cohen_kappa_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

import pandas as pd
import numpy as np

In [2]:
voice_data = pd.read_csv('./data/voiceDataSet.csv')

In [3]:
voice_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   meanfreq  3168 non-null   float64
 1   sd        3168 non-null   float64
 2   median    3168 non-null   float64
 3   Q25       3168 non-null   float64
 4   Q75       3168 non-null   float64
 5   IQR       3168 non-null   float64
 6   skew      3168 non-null   float64
 7   kurt      3168 non-null   float64
 8   sp.ent    3168 non-null   float64
 9   sfm       3168 non-null   float64
 10  mode      3168 non-null   float64
 11  centroid  3168 non-null   float64
 12  meanfun   3168 non-null   float64
 13  minfun    3168 non-null   float64
 14  maxfun    3168 non-null   float64
 15  meandom   3168 non-null   float64
 16  mindom    3168 non-null   float64
 17  maxdom    3168 non-null   float64
 18  dfrange   3168 non-null   float64
 19  modindx   3168 non-null   float64
 20  label     3168 non-null   obje

In [4]:
voice_data.isna().sum()

meanfreq    0
sd          0
median      0
Q25         0
Q75         0
IQR         0
skew        0
kurt        0
sp.ent      0
sfm         0
mode        0
centroid    0
meanfun     0
minfun      0
maxfun      0
meandom     0
mindom      0
maxdom      0
dfrange     0
modindx     0
label       0
dtype: int64

In [5]:
voice_data.sample(5)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
850,0.178254,0.053752,0.190419,0.117253,0.219028,0.101776,1.912809,6.834770,0.891691,0.296863,...,0.178254,0.114361,0.047244,0.277457,1.104226,0.023438,5.671875,5.648438,0.110749,male
1896,0.192050,0.069854,0.216128,0.178150,0.241332,0.063181,1.762694,5.862619,0.922779,0.546754,...,0.192050,0.184095,0.016754,0.266667,0.316761,0.054688,0.781250,0.726562,0.391193,female
2844,0.224698,0.033298,0.226417,0.218011,0.237974,0.019962,3.128295,13.756715,0.791817,0.184991,...,0.224698,0.189976,0.054237,0.269663,1.580915,0.234375,8.484375,8.250000,0.135866,female
1938,0.166078,0.091112,0.207214,0.074179,0.236816,0.162637,4.237644,41.155866,0.908195,0.499168,...,0.166078,0.188951,0.027634,0.275862,0.182566,0.007812,0.695312,0.687500,0.339394,female
2717,0.196093,0.045943,0.202180,0.166703,0.224305,0.057602,1.457014,5.147233,0.909567,0.324592,...,0.196093,0.175814,0.047809,0.279070,1.394231,0.023438,8.812500,8.789062,0.114753,female


In [6]:
voice_data['label'] = voice_data['label'].apply(lambda x: 1 if x == 'male' else 0)

In [7]:
X = voice_data.drop(columns=['label'])
y = voice_data['label']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [9]:
# scaler = StandardScaler()

# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

In [10]:
pipe = Pipeline([
    ('scaling', StandardScaler()),
    ('regression', LogisticRegression())
])

In [11]:
model = pipe.fit(X_train, y_train)

y_pred = model.predict(X_test)

np.round(accuracy_score(y_test, y_pred), 3)

0.973

## Практика 2 (алгоритм kNN (метод ближайших соседей)):

Теперь познакомимся с новым алгоритмом классификации, а также снова потренируемся делить выборку и осуществлять кросс-валидацию. Кроме того, посмотрим, как можно оценить качество классификации для случая, когда наша классификация не бинарная, т.е. у нас несколько классов.

В этом кейсе мы попробуем определять типы стекла по его характеристикам.

Откроем данные и увидим, что в первых столбцах показатели содержания различных веществ в стекле, а в последнем (Type) — непосредственно тип стекла.

In [12]:
glass_data = pd.read_csv('./data/glass.csv')

In [13]:
glass_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   RI      214 non-null    float64
 1   Na      214 non-null    float64
 2   Mg      214 non-null    float64
 3   Al      214 non-null    float64
 4   Si      214 non-null    float64
 5   K       214 non-null    float64
 6   Ca      214 non-null    float64
 7   Ba      214 non-null    float64
 8   Fe      214 non-null    float64
 9   Type    214 non-null    int64  
dtypes: float64(9), int64(1)
memory usage: 16.8 KB


In [14]:
glass_data.sample(5)

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
175,1.52119,12.97,0.33,1.51,73.39,0.13,11.27,0.00,0.28,5
189,1.52365,15.79,1.83,1.31,70.43,0.31,8.61,1.68,0.00,7
153,1.51610,13.42,3.40,1.22,72.69,0.59,8.32,0.00,0.00,3
52,1.51808,13.43,2.87,1.19,72.84,0.55,9.03,0.00,0.00,1
80,1.51592,12.86,3.52,2.12,72.66,0.69,7.97,0.00,0.00,2


Наша классификация мультиклассовая, поэтому мы должны посмотреть, сколько у нас классов. Если вы вообще ничего не знаете о задаче в плане количества классов, обязательно начинайте с этого, поскольку это повлияет на ход решения. Особенно важно понять, бинарная классификация или нет.

Сколько классов стекла представлено в этой задаче?

In [15]:
len(glass_data['Type'].unique())

6

Итак, мы выяснили, сколько у нас классов и узнали, что в этот раз у нас будет не бинарная классификация. А значит, мы не сможем использовать некоторые метрики качества (например, **precision** и **recall**).

Приступим непосредственно к построению модели. На примере этой задачи мы узнаем новый алгоритм **kNN (метод ближайших соседей)**. Это один из простейших методов классификации. 

Его называют ленивым классификатором, потому что во время обучения модели он ничего не делает, просто считывает и сохраняет тренировочные данные. Сама классификация для него начинается тогда, когда ему дают тестовые данные. Тогда kNN проходит два базовых шага:

1. Сначала он ищет k ближайших размеченных точек данных – эти точки и называют k ближайшими соседями.
2. Затем, используя классы соседей, kNN решает, как лучше классифицировать новые данные.

> #### Пример
Предположим, что алгоритм хочет классифицировать вас по профессии. Он видит, что люди, рядом с которыми вы находитесь (родители, друзья), — врачи. Тогда он решает, что вы — тоже скорее всего врач. Грубо говоря, он основывается на принципе, что подобное находится рядом с подобным.

А что делать, если ближайшие соседи не принадлежат одному классу? Можно решать по принципу большинства или дать каждому соседу вес (в зависимости от удаленности), с которым он будет вносить вклад в решение.

Например, попробуем классифицировать зеленый кружочек. На этом примере мы видим, что выбор числа соседей (это настраиваемый параметр) очень важен.

![kNN](./img/kNN.png)

Если мы будем рассматривать трех соседей, то кружочек будет причислен к классу треугольников. А если пять соседей — к классу квадратов.

> Важно! Количество соседей должно быть нечетное во избежание спорной ситуации.

Итак, мы разобрались с новым алгоритмом. Теперь воспользуемся `k-fold` валидацией на пяти разбиениях и обучим модель:

In [16]:
X = glass_data.drop(columns=['Type'])
y = glass_data['Type']

In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold

model = KNeighborsClassifier(n_neighbors=3)
kf = KFold(n_splits=5)
cross_val_score(model, X, y, cv=kf, scoring="accuracy")

array([0.44186047, 0.65116279, 0.3255814 , 0.34883721, 0.07142857])

Если мы посмотрим на результаты классификации, то увидим достаточно большой разброс значений точности, а также в целом невысокие значения.

Это произошло в силу очень маленькой выборки в условиях большого количества классов (6 классов и около 200 наблюдений — крайне мало для получения высокого качества модели). Также плохой результат возможен по причине использования довольно простого алгоритма.

### Задание

> Измените количество разбиений на 10. Вычислите среднее значение метрики **accuracy** по 10 разбиениям и введите ниже, округлите до сотых.

In [18]:
model = KNeighborsClassifier(n_neighbors=3)
kf = KFold(n_splits=10)
np.round(cross_val_score(model, X, y, cv=kf, scoring="accuracy").mean(), 2)

0.53

## Практика 3

В следующей задаче мы будем **диагностировать болезни сердца** по различным медицинским параметрам пациентов.

Избавьтесь от выбросов, чтобы они не ухудшили качество нашей модели. Для того, чтобы избавиться от выбросов, уберите все данные, значения признаков которых отстают от первого или третьего квартиля более, чем на 1.5 межквартильного размаха.

In [19]:
heart_fin_data = pd.read_csv('./data/heart_fin1.csv', sep=';')

In [20]:
heart_fin_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 303 entries, 0 to 302
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       303 non-null    int64  
 1   sex       303 non-null    int64  
 2   cp        303 non-null    int64  
 3   trestbps  303 non-null    int64  
 4   chol      303 non-null    int64  
 5   restecg   303 non-null    int64  
 6   thalach   303 non-null    int64  
 7   exang     303 non-null    int64  
 8   oldpeak   303 non-null    float64
 9   target    303 non-null    int64  
dtypes: float64(1), int64(9)
memory usage: 23.8 KB


In [21]:
heart_fin_data.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
restecg     0
thalach     0
exang       0
oldpeak     0
target      0
dtype: int64

In [22]:
def get_statistical_borders(df, column):
    perc25 = df[column].quantile(q=0.25, interpolation='midpoint')
    perc75 = df[column].quantile(q=0.75, interpolation='midpoint')
    IQR = perc75 - perc25

    take_from = perc25 - 1.5*IQR
    take_to = perc75 + 1.5*IQR

    return take_from, take_to

In [23]:
for column in heart_fin_data.columns:
    take_from, take_to = get_statistical_borders(heart_fin_data, column)
    heart_fin_data = heart_fin_data[(heart_fin_data[column] >= take_from) & (heart_fin_data[column] <= take_to)]

Сколько наблюдений осталось после удаления выбросов?

In [24]:
len(heart_fin_data)

284

Теперь разбейте выборку на тестовую и обучающую с параметрами `test_size=0.15, random_state=5`.

Обучите модели логистической регрессии (c параметром `max_iter=1000`) и KNN (с количеством соседей, равным 3) на этих данных. Вычислите метрики качества.

У какой модели выше значение ROC AUC?

In [25]:
X = heart_fin_data.drop(columns=['target'])
y = heart_fin_data['target']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=5)

In [27]:
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [28]:
kneighbors_model = KNeighborsClassifier(n_neighbors=3)
kneighbors_model.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [29]:
from sklearn.metrics import roc_auc_score

**ROC AUC** считается не по меткам предсказанным вероятностям. Для этого используйте predict_proba() (нужен второй столбец)

In [30]:
log_y_pred_proba = log_model.predict_proba(X_test)

kneighbors_y_pred_proba = kneighbors_model.predict_proba(X_test)

print('Logistic Regression: {}\nKNeighbors: {}'.format(
    np.round(roc_auc_score(y_test, log_y_pred_proba[:, 1]), 2),
    np.round(roc_auc_score(y_test, kneighbors_y_pred_proba[:, 1]), 2)
))

Logistic Regression: 0.82
KNeighbors: 0.65
